# 단일 분류기 (Single Classifier) 학습

계층형 분류기와의 성능 비교를 위해, 모든 서브레딧을 한 번에 분류하는 단일 분류기 모델을 학습합니다. 이 모델은 9개의 서브레딧을 직접적으로 구분하는 작업을 수행합니다.

## 학습 과정
1.  **데이터 준비**: 전처리된 텍스트 데이터를 `TfidfVectorizer`를 사용하여 고차원 벡터로 변환합니다.
2.  **모델 선택**: 선형 모델인 `SGDClassifier`를 기본 분류기로 사용합니다. 이 모델은 대용량 텍스트 데이터에 효율적입니다.
3.  **하이퍼파라미터 최적화**: `RandomizedSearchCV`를 이용하여 최적의 하이퍼파라미터를 탐색합니다. 이를 통해 모델의 일반화 성능을 높입니다.
4.  **모델 학습 및 평가**: 최적화된 하이퍼파라미터로 모델을 학습시킨 후, 테스트 데이터셋으로 최종 성능을 평가합니다.

In [8]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 필요한 라이브러리 임포트
import pandas as pd

In [10]:
FILE_PATH = 'assets/' 
MODEL_PATH = 'models/'

# 서브레딧 및 그룹 정의
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries',
    'learnprogramming', 'cscareerquestions', 'SideProject',
    'TrueFilm', 'booksuggestions', 'TrueGaming'
]
GROUP_MAP = {'Mystery': SUBREDDITS[0:3], 'Dev': SUBREDDITS[3:6], 'Culture': SUBREDDITS[6:9]}
VECTOR_DIMENSION = 5000  # 문서 벡터 차원

In [11]:
preprocessed_df = pd.read_csv(FILE_PATH + 'reddit_posts_preprocessed.csv')

preprocessed_df.head(5)

,subreddit,preprocessed_content
0,Thetruthishere,truth leviathan leviathan isnt scary monster a...
1,Thetruthishere,streetlight front house turns every time walk ...
2,Thetruthishere,reflection window seconds behind nearly empty ...
3,Thetruthishere,strange knocking empty apartment night ive liv...
4,Thetruthishere,whats unexplainable thing youve ever witnessed...


In [12]:
# 전체 서브레딧을 대상으로 하는 단일 분류기 학습
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import loguniform

# 벡터화
# NaN 값을 빈 문자열로 대체
preprocessed_df['preprocessed_content'] = preprocessed_df['preprocessed_content'].fillna('')
preprocessed_df.head()
vectorizer = TfidfVectorizer(max_features=VECTOR_DIMENSION, ngram_range=(1, 2))
X = vectorizer.fit_transform(preprocessed_df['preprocessed_content']).toarray()

# 서브레딧 라벨링
y_subreddit = preprocessed_df['subreddit']

# 데이터 분할 (Train / Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_subreddit, test_size=0.2, random_state=42, stratify=y_subreddit
)
print(f"데이터 분할 완료: 학습용 {X_train.shape[0]}개, 테스트용 {X_test.shape[0]}개")

# RandomizedSearchCV 설정
param_dist = {
    'loss': ['log_loss'],  
    'alpha': loguniform(1e-5, 1e-1),
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000, 3000]
}
base_model = SGDClassifier(random_state=42, class_weight='balanced', n_jobs=1)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
random_search_single = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,          
    cv=cv_strategy,     
    scoring='accuracy', 
    n_jobs=-1,          
    verbose=1,
    random_state=42
)
print("단일 분류기(Single Classifier) 최적화 시작 (RandomizedSearch)...")
random_search_single.fit(X_train, y_train)

# 결과 출력 및 최종 평가
print("\n" + "="*50)
print(f"최적의 하이퍼파라미터: {random_search_single.best_params_}")
print(f"교차 검증 최고 점수 (Best CV Score): {random_search_single.best_score_:.4f}")

# 최적의 모델 추출
single_classifier = random_search_single.best_estimator_

# 최종 테스트 데이터로 객관적 성능 평가
y_pred = single_classifier.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)
print("-" * 50)
print(f"최종 테스트 정확도 (Test Accuracy): {final_accuracy:.4f}")
print("="*50)
print("\n[상세 분류 리포트]")
print(classification_report(y_test, y_pred))


데이터 분할 완료: 학습용 7065개, 테스트용 1767개
단일 분류기(Single Classifier) 최적화 시작 (RandomizedSearch)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits

최적의 하이퍼파라미터: {'alpha': np.float64(7.068974950624602e-05), 'loss': 'log_loss', 'max_iter': 1000, 'penalty': 'l2'}
교차 검증 최고 점수 (Best CV Score): 0.9127
--------------------------------------------------
최종 테스트 정확도 (Test Accuracy): 0.9106

[상세 분류 리포트]
                      precision    recall  f1-score   support

Glitch_in_the_Matrix       0.88      0.83      0.85       198
         SideProject       0.91      0.95      0.93       189
      Thetruthishere       0.81      0.86      0.83       190
            TrueFilm       0.97      0.93      0.95       198
          TrueGaming       0.96      0.97      0.97       198
 UnresolvedMysteries       0.98      0.98      0.98       198
     booksuggestions       0.96      0.98      0.97       200
   cscareerquestions       0.84      0.87      0.86       197
    learnprogramming       0.88      0.81 

In [14]:
# 단일 분류기 모델 저장
import joblib

joblib.dump(single_classifier, MODEL_PATH + 'single_classifier.pkl')
print(f"단일 분류기 모델이 '{MODEL_PATH}single_classifier.pkl'에 저장되었습니다.")

단일 분류기 모델이 'models/single_classifier.pkl'에 저장되었습니다.
